# Import Relevant Libraries

In [ ]:
#This is my first ML Solo proyect.  I'll appreciate some advices or corrections you can give me. 
# I'm concern about the Durbin-Watson and the Cond.No but I couldn't fix it so any ideas will be extremly heplful and welcomed. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

### Loading the Raw Data

In [ ]:
raw_data = pd.read_csv("../input/suicide-rates-overview-1985-to-2016/master.csv")
raw_data.describe(include='all')

## Preprocessing

In [ ]:
#Drop unnecesary features, country-year because is repeated and HDI because is not enought. 
data= raw_data.drop(['country-year', 'HDI for year'], axis=1)
data.describe(include='all')

In [ ]:
#No missing values
data.isnull().sum()

### Changing col names

In [ ]:
data.rename(columns={'population':'pop', 'gdp_per_capita ($)':'gdp_per_capita', ' gdp_for_year ($) ':'gdp_for_year'}, inplace=True)
data.gdp_for_year = data.gdp_for_year.apply(lambda x: float(''.join(x.split(','))))
data.age = data.age.apply(lambda x: x.replace("years", ""))
data.describe(include='all')

## Plot the Data
### By Country

In [ ]:
#Creating a col % of population comited suicide
data['suicides/pop'] = data['suicides_no']*100/data['pop']
data_suicides_country = data.groupby('country', as_index=False)['suicides/pop'].mean().sort_values(by='suicides/pop', ascending=False)

data_suicides_country

In [ ]:
plt.figure(figsize=(8,32))
sns.barplot(y=data_suicides_country.country, x=data_suicides_country['suicides/pop'])
plt.xlabel("suicides / population")
plt.ylabel("country")
plt.title("suicides/pop vs country (1985-2016)")
plt.show()

### By Gender / Pop / Year

In [ ]:
#Cheking the difference between men and women over the years
plt.figure(figsize=(9,6))
plt.title('Suicide number by Gender (1985-2016)', fontsize=20)
data_men = data[data['sex'] == 'male']
data_women = data[data['sex'] == 'female']
sns.lineplot(data_men.year, data_men.suicides_no, ci = None)
sns.lineplot(data_women.year, data_women.suicides_no, ci = None)
plt.legend(['men','women'])
plt.show()

## By Age

In [ ]:
data_age = data.groupby(['year', 'age'])['suicides_no', 'pop'].sum()
data_reset = data_age.copy().reset_index()
plt.figure(figsize=(9,6))
plt.title('Suicide number by Age (1985-2016)', fontsize=20)
sns.lineplot('year', data_reset.suicides_no*100/data_reset['pop'], hue='age',
            data=data_reset, linewidth=2.5, style='age')
plt.show()

### By Generation

In [ ]:
data_generation =  data.groupby('generation', as_index=False)['suicides_no'].sum().sort_values(by='suicides_no', ascending=False)
data_generation 

In [ ]:
plt.figure(figsize=(6,6))
plt.title('Suicide number (1985-2016)', fontsize=20)
plt.pie(data_generation.suicides_no, explode =(0.05, 0.05, 0.05, 0.05, 0.05, 0.05), labels=data_generation.generation, autopct='%1.1f%%')
plt.show()

## Exploring PDF's

In [ ]:
# suicide_no, gdp_per_capita, suicides/100k pop and pop have Outliers
# Convert sex, age and generation into dummy variables
data.describe(include='all')

## Dealing with Outliers

In [ ]:
q1 = data['suicides_no'].quantile(0.99)
data_1 = data[data['suicides_no']<q1]

q2 = data_1['gdp_per_capita'].quantile(0.99)
data_2 = data_1[data['gdp_per_capita']<q2]


q3 = data_2['suicides/100k pop'].quantile(0.99)
data_3 = data_2[data['suicides/100k pop']<q3]

q4 = data_3['pop'].quantile(0.99)
data_4 = data_3[data['pop']<q4]

In [ ]:
f, axes = plt.subplots(1, 4, figsize= (25,6))

sns.distplot(data_1['suicides_no'], ax=axes[0])
sns.distplot(data_2['gdp_per_capita'], ax=axes[1])
sns.distplot(data_3['suicides/100k pop'], ax=axes[2])
sns.distplot(data_4['pop'], ax=axes[3])

plt.show()

In [ ]:
#Reset index of values left and drops the old index
#cln = cleaned
data_cln = data_4.reset_index(drop=True)
data_cln.describe(include='all')

## Cheking the OLS assumptins

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize= (25,6))
ax1.scatter(data_cln['pop'], data_cln['suicides_no'], alpha=0.2)
ax1.set_title('population and suicides_no')
ax2.scatter(data_cln['suicides/100k pop'], data_cln['suicides_no'], alpha=0.2)
ax2.set_title('suicides/100k pop and suicides_no')
ax3.scatter(data_cln['gdp_per_capita'], data_cln['suicides_no'], alpha=0.2)
ax3.set_title('gdp_per_capita ($) and suicides_no')

plt.show()

## Relaxing Assumptions

In [ ]:
data_cln['log pop'] = np.log(data_cln['pop'].replace(0,np.nan))
data_cln['log suicides/100k pop'] = np.log(data_cln['suicides/100k pop'].replace(0,np.nan))
data_cln['log gdp_per_capita'] = np.log(data_cln['gdp_per_capita'].replace(0,np.nan))
data_cln['log suicides_no'] = np.log(data_cln['suicides_no'].replace(0,np.nan))

f, axes = plt.subplots(1, 4, figsize= (25,6))

sns.distplot(data_cln['log suicides_no'], ax=axes[0])
sns.distplot(data_cln['log pop'], ax=axes[1])
sns.distplot(data_cln['log gdp_per_capita'], ax=axes[2])
sns.distplot(data_cln['log suicides/100k pop'], ax=axes[3])

plt.show()

In [ ]:
data_cln.describe(include='all')

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize= (25,6))
ax1.scatter(data_cln['log pop'], data_cln['suicides_no'], alpha=0.2)
ax1.set_title('log population and log suicides_no')
ax2.scatter(data_cln['log suicides/100k pop'], data_cln['suicides_no'], alpha=0.2)
ax2.set_title('log suicides/100k pop and log suicides_no')
ax3.scatter(data_cln['log gdp_per_capita'], data_cln['suicides_no'], alpha=0.2)
ax3.set_title('log gdp_per_capita ($) and suicides_no')

plt.show()

## Dummy Variables

In [ ]:
data_dummy = data_cln.drop(['country'], axis=1)
data_dummy = pd.get_dummies(data_dummy, drop_first=True)
data_dummy.columns.values

### Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
variables = data_dummy.drop(['suicides_no', 'suicides/100k pop', 'gdp_for_year', 'log pop', 'log suicides/100k pop', 'log gdp_per_capita', 'log suicides_no'], axis=1)

vif['VIF'] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif['Features'] = variables.columns
vif

In [ ]:
# Drop year beacuse it's over 10
# Drop all features that won't use because we have the transformed features
data_preprocessed = data_dummy.drop(['suicides/100k pop', 'pop', 'gdp_per_capita', 'log suicides_no', 'year', 'log suicides/100k pop',], axis=1)
data_preprocessed.describe(include='all').T

## Multiple Linear Regression
### Declare inputs and targets


In [ ]:
targets = data_preprocessed['suicides_no']

inputs = data_preprocessed.drop(['suicides_no'], axis=1)
inputs.columns.values

### Scale the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

inputs_scaled = scaler.transform(inputs)

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

## Create Regression

In [ ]:
reg = LinearRegression()
reg.fit(x_train, y_train)

In [ ]:
y_hat = reg.predict(x_train)

In [ ]:
#The errors are normally distribuited
sns.distplot(y_train - y_hat)
plt.title("Residuals PDF", size=18)

In [ ]:
# We're trying to predict human behavior so we can expect R squares less than 50%
reg.score(x_train,y_train)

### Finding Weights and Bias

In [ ]:
#bias
reg.intercept_

In [ ]:
#features weights
reg.coef_

In [ ]:
reg_summary = pd.DataFrame(inputs.columns.values, columns=['Features'])
reg_summary['Weights'] = reg.coef_
df_bias = pd.DataFrame(['bias'], columns=['Features'])
df_bias['Weights'] = reg.intercept_
reg_summary = reg_summary.append(df_bias, ignore_index=True)
reg_summary


In [ ]:
#male sex has more weight on the prediction and it's the benchmark
data_cln.sex.unique()

In [ ]:
# age-15-24 is the benchmark
data_cln.age.unique()

In [ ]:
#Boomers are the benchmark
data_cln.generation.unique()

## Testing

In [ ]:
y_hat_test = reg.predict(x_test)
df_predict = pd.DataFrame(y_hat_test, columns=['Prediction'])

y_test = y_test.reset_index(drop=True)
df_predict['Target'] = y_test
df_predict

In [ ]:
# We have almost same means between Prediction and Target and the standard deviation si not so different
#
df_predict['Residual'] = df_predict['Target'] - df_predict['Prediction']
df_predict['Difference%'] = np.absolute(df_predict['Residual']/df_predict['Target']*100)
df_predict.describe()

In [ ]:
#Drop this features because they are not useful
inputs_new = data_preprocessed.drop(['suicides_no', 'age_55-74 ', 'generation_G.I. Generation'], axis=1)

results = sm.OLS(targets,inputs_new).fit()
results.summary()
# By the Durbin_watson > 2 we can see there is some Possitive Autocorrelation